In [3]:
import numpy as np 
import pandas as pd 
!pip install xlrd
import re
!pip install PyEnchant

  Using cached https://files.pythonhosted.org/packages/9e/54/04d88a59efa33fefb88133ceb638cdf754319030c28aadc5a379d82140ed/pyenchant-2.0.0.tar.gz
    Complete output from command python setup.py egg_info:
    Traceback (most recent call last):
      File "<string>", line 1, in <module>
      File "C:\Users\JUSTIN~1\AppData\Local\Temp\pip-install-yku4rsdm\PyEnchant\setup.py", line 212, in <module>
        import enchant
      File "C:\Users\JUSTIN~1\AppData\Local\Temp\pip-install-yku4rsdm\PyEnchant\enchant\__init__.py", line 92, in <module>
        from enchant import _enchant as _e
      File "C:\Users\JUSTIN~1\AppData\Local\Temp\pip-install-yku4rsdm\PyEnchant\enchant\_enchant.py", line 145, in <module>
        raise ImportError(msg)
    ImportError: The 'enchant' C library was not found. Please install it via your OS package manager, or use a pre-built binary wheel from PyPI.
    
    ----------------------------------------


Command "python setup.py egg_info" failed with error code 1 in C:\Users\JUSTIN~1\AppData\Local\Temp\pip-install-yku4rsdm\PyEnchant\


### English Word Test

In [6]:
with open("./Data Files/EN_US.txt") as word_file:
    english_words = set(word.strip().lower() for word in word_file)
def is_word(word):
    return word.lower() in english_words

206 ns ± 2.52 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [2]:
excel_file = './Data Files/UBS Statement.xlsx'

In [ ]:
def uniform_solution_solver(excel_file):
    df = pd.read_excel(excel_file)
    
    
    #easy rows
    #CcyAccountCode should have curency column
    output_col = ['Action','CcyAccountCode','TradeDate','ValueDate','Narration','Amount','TransactionType','Ticker','Quantity','Price','DirtyPrice','HashTags']
    output_df = pd.DataFrame(columns=output_col)
    output_df['TradeDate'] = df['Trade date']
    output_df['ValueDate'] = df['Value date']
    output_df['Narration'] = df['Information']
    output_df['Amount'] = -df['Debit']
    output_df['Amount'] = output_df['Amount'].add(df['Credit'],fill_value = 0)
    output_df.drop(output_df.index[output_df['Amount'].isna()],inplace=True)
    output_df.index = range(len(output_df))
    output_df['Action'] = pd.Series(['CreateTransaction']*len(output_df))
    
    #CcyAccountCode
    ccyaccountcode = 'Dummy Account Number'
    output_df['CcyAccountCode'] = pd.Series([ccyaccountcode]*len(output_df))
    
    #detect narration type
    narration = []
    ticker_boo = []
    ticker_info = []
    for i in output_df['Narration']:
        if type(i) is str:
            narration.append(text_classification(i)[0])
            if len(text_classification(i)[1]) != 0:
                ticker_boo.append(['LoanRef',True])
                ticker_info.append(text_classification(i)[1])
            else:
                ticker_boo.append(['No ticker',False])
                ticker_info.append(np.nan)
        else:
            pass
    output_df['TransactionType'] = pd.Series(narration)
    #ticker
    output_df['Ticker'] = pd.Series(ticker_boo)
    for i in range(len(output_df['Ticker'])):
        if output_df['Ticker'][i][1]:
            output_df['Ticker'][i] = '_'.join((output_df['Ticker'][i][0],output_df['CcyAccountCode'][i],' '.join(ticker_info[i])))
        elif ~output_df['Ticker'][i][1]:
            output_df['Ticker'][i] = np.nan
        else:
            print("Please check code. Error.")
    
    
    return output_df

In [ ]:
print(uniform_solution_solver(excel_file))

In [ ]:
def text_classification(string):
    splitted = string.split(' ')
    output = []
    ticker = False
    for splits in splitted:
        if splits.lower() == 'repayment':
            output.append('LoanRepay')
            ticker = True
        elif splits.lower() == 'interest':
            output.append('LoanCost')
            ticker = True
        elif splits.lower() == 'loan':
            output.append('LoanCreate')
            ticker = True
        elif splits.lower() == 'dividend':
            output.append('Distribution')
        elif splits.lower() == 'investment':
            output.append('Purchase')
        else:
            pass
    no_char = ''
    if ticker:
        no_char = re.sub('[a-z,A-Z]', '',string)
    if len(output) == 1:
        return (output[0],[i for i in no_char.split(' ') if i != ''])
    else:
        return 'Please Check Manually'